In [ ]:
!pip install tensorflow pandas numpy scikit-learn scikit-image click

In [ ]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import skimage.io
import sklearn.preprocessing
import sklearn.model_selection
from tqdm import tqdm

DIR = "/content/drive/My Drive/leafnet_thesis/leaf_valid_v3"
CLASSES = sorted(os.listdir(DIR))

def load_model():
    return tf.keras.models.load_model("/content/drive/My Drive/leafnet_thesis/models/leafnet_v3.h5", compile=True)

def predict_image(image_file):
    model = load_model()
    data = pd.DataFrame({"x": [image_file]})
    gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_dataframe(data, x_col="x", shuffle=False, target_size=(256, 256), class_mode=None, color_mode="rgb")
    pred = model.predict(gen)
    print(pred)
    print(CLASSES[np.argmax(pred[0])])
    print(f"{np.max(100*pred):.2f}%")

def predict_class(class_directory, classes):
    model = load_model()
    classname = os.path.basename(class_directory)
    data = {"x": [], "y": []}
    for image in os.listdir(class_directory):
        data["x"].append(os.path.join(class_directory, image))
        data["y"].append(classname)
    data = pd.DataFrame(data)
    batch_size = min(8, data.shape[0])
    gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_dataframe(
        data,
        x_col="x",
        y_col="y",
        shuffle=False,
        target_size=(256, 256),
        classes=classes,
        batch_size=batch_size,
    )
    score = model.evaluate(gen, workers=-1, steps=data.shape[0] // batch_size, max_queue_size=batch_size, verbose=0)
    print(f"Score for {classname}: {score[1] * 100:.2f} %")
    return {"loss": score[0], "accuracy": score[1]}

def predict_classes(directory):
    results = {"class": [], "loss": [], "accuracy": []}
    classes = sorted(os.listdir(directory))
    total = 0
    for class_directory in classes:
        result = predict_class(os.path.join(directory, class_directory), classes)
        results["class"].append(class_directory)
        results["loss"].append(result["loss"])
        results["accuracy"].append(result["accuracy"])
        total += len(os.listdir(os.path.join(directory, class_directory)))
    model = load_model()
    gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(
        directory,
        shuffle=False,
        target_size=(256, 256),
        color_mode="rgb",
        batch_size=8,
    )
    score = model.evaluate(gen, workers=-1, steps=total // 8, max_queue_size=8, verbose=0)
    results["class"].append("TOTAL")
    results["loss"].append(score[0])
    results["accuracy"].append(score[1])
    print(pd.DataFrame(results))
    pd.DataFrame(results).to_csv("/content/drive/My Drive/leafnet_thesis/acc_dur.csv", index=False)

def predict_directory(directory):
    model = load_model()
    gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(directory, shuffle=False, target_size=(256,256), color_mode="rgb", batch_size=8)
    score = model.evaluate(gen, workers=-1, steps=502//8, max_queue_size=8)
    print(score)

# predict_directory("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3")
# predict_image("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3/Grape___Black_rot/198.jpg")
# predict_class("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3/Tomato___Spider_mites Two-spotted_spider_mite")
predict_classes("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3")

Found 44 validated image filenames belonging to 33 classes.
Score for Apple___Apple_scab: 92.50 %
Found 49 validated image filenames belonging to 33 classes.
Score for Apple___Black_rot: 97.92 %
Found 28 validated image filenames belonging to 33 classes.
Score for Apple___Cedar_apple_rust: 100.00 %
Found 50 validated image filenames belonging to 33 classes.


KeyboardInterrupt: ignored

In [ ]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import skimage.io
import sklearn.preprocessing
import sklearn.model_selection
from tqdm import tqdm
from click import secho

DIR = "/content/drive/My Drive/leafnet_thesis/leaf_valid_v2"
CLASSES = sorted(os.listdir(DIR))

def load_model():
    return tf.keras.models.load_model("/content/drive/My Drive/leafnet_thesis/models/leafnet_v3.h5", compile=True)

def predict_image(image_file):
    model = load_model()
    data = pd.DataFrame({"x": [image_file]})
    gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_dataframe(data, x_col="x", shuffle=False, target_size=(256, 256), class_mode=None, color_mode="rgb")
    pred = model.predict(gen)
    print(pred)
    print(CLASSES[np.argmax(pred[0])])
    print(f"{np.max(100*pred):.2f}%")

def predict_class(class_directory, classes):
    model = load_model()
    classname = os.path.basename(class_directory)
    data = {"x": [], "y": []}
    for image in sorted(os.listdir(class_directory)):
        data["x"].append(os.path.join(class_directory, image))
        data["y"].append(classname)
    data = pd.DataFrame(data)
    batch_size = min(32, data.shape[0])
    gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_dataframe(
        data,
        x_col="x",
        y_col="y",
        shuffle=False,
        target_size=(256, 256),
        classes=classes,
        batch_size=batch_size,
    )
    predictions = model.predict(gen, workers=-1, steps=data.shape[0] // batch_size, max_queue_size=batch_size, verbose=0)
    truth_prediction = {"image": [], "truth": [], "prediction": []}
    for image, classname, prediction in zip(data["x"], data["y"], predictions):
        truth_prediction["image"].append(image)
        truth_prediction["truth"].append(classname)
        truth_prediction["prediction"].append(classes[np.argmax(prediction)])
    new_data = pd.DataFrame(truth_prediction)
    return new_data

def predict_classes(directory):
    results = {"class": [], "loss": [], "accuracy": []}
    classes = sorted(os.listdir(directory))
    total = 0
    results = pd.DataFrame({"image": [], "truth": [], "prediction": []})
    for class_directory in classes:
        result = predict_class(os.path.join(directory, class_directory), classes)
        results = results.append(result, ignore_index=True)
    print(results)
    results.to_csv("results.csv", index=False)
    for _, row in results.iterrows():
        if row["truth"] == row["prediction"]:
            secho(f"{row['image']}: {row['truth']}", fg="green")
        else:
            secho(f"{row['image']}: {row['truth']} vs {row['prediction']}", fg="red")

def predict_directory(directory):
    model = load_model()
    gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(directory, shuffle=False, target_size=(256,256), color_mode="rgb", batch_size=32)
    score = model.evaluate(gen, workers=-1, steps=986//32, max_queue_size=32)
    print(score)

# predict_directory("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3")
# predict_image("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3/Grape___Black_rot/198.jpg")
# predict_class("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3/Tomato___Spider_mites Two-spotted_spider_mite")
predict_classes("/content/drive/My Drive/leafnet_thesis/leaf_valid_v2")

Found 27 validated image filenames belonging to 33 classes.
Found 36 validated image filenames belonging to 33 classes.
Found 42 validated image filenames belonging to 33 classes.
Found 26 validated image filenames belonging to 33 classes.
Found 31 validated image filenames belonging to 33 classes.
Found 36 validated image filenames belonging to 33 classes.
Found 25 validated image filenames belonging to 33 classes.
Found 39 validated image filenames belonging to 33 classes.
Found 30 validated image filenames belonging to 33 classes.
Found 27 validated image filenames belonging to 33 classes.
Found 27 validated image filenames belonging to 33 classes.
Found 20 validated image filenames belonging to 33 classes.
Found 32 validated image filenames belonging to 33 classes.
Found 31 validated image filenames belonging to 33 classes.
Found 27 validated image filenames belonging to 33 classes.
Found 27 validated image filenames belonging to 33 classes.
Found 30 validated image filenames belon

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import skimage.io
import sklearn.preprocessing
import sklearn.model_selection
from tqdm import tqdm
from PIL import Image
from time import time

DIR = "/content/drive/My Drive/leafnet_thesis/leaf_valid_v3"
CLASSES = sorted(os.listdir(DIR))
SIZE = (256, 256)

def load_model():
    return tf.keras.models.load_model("/content/drive/My Drive/leafnet_thesis/models/leafnet_v3.h5", compile=True)

model = load_model()

def predict_image(image_file):
    image = Image.open(image_file).convert("RGB").resize(SIZE, Image.ANTIALIAS)
    image = np.asarray(image)
    image = image.astype('float32')
    image /= 255.0
    start = time()
    result = model.predict(np.array([image]))[0]
    duration = time() - start
    return np.argmax(result), duration

def predict_class(class_directory, classes):
    classname = os.path.basename(class_directory)
    images = []
    for image in os.listdir(class_directory):
        images.append({"x": os.path.join(class_directory, image), "y": classname})
    correct = 0
    total_duration = 0
    for image in images:
        result, duration = predict_image(image["x"])
        if classes[result] == classname:
            correct += 1
        total_duration += duration
    score = correct / len(images)
    print(f"Score for {classname}: {score * 100:.2f} %.")
    print(f"Done in {total_duration:.3f} seconds for {len(images)} images.")
    return {"accuracy": score, "duration": total_duration}

def predict_classes(directory):
    results = {"class": [], "duration": [], "accuracy": []}
    classes = sorted(os.listdir(directory))
    total = 0
    for class_directory in classes:
        result = predict_class(os.path.join(directory, class_directory), classes)
        results["class"].append(class_directory)
        results["duration"].append(result["duration"])
        results["accuracy"].append(result["accuracy"])
        total += len(os.listdir(os.path.join(directory, class_directory)))
    print(pd.DataFrame(results))
    pd.DataFrame(results).to_csv("/content/drive/My Drive/leafnet_thesis/acc_loss.csv", index=False)

def predict_directory(directory):
    model = load_model()
    gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(directory, shuffle=False, target_size=(256,256), color_mode="rgb", batch_size=8)
    score = model.evaluate(gen, workers=-1, steps=1436//8, max_queue_size=8)
    print(score)

# predict_directory("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3")
# predict_image("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3/Apple___Apple_scab/1001.jpg")
# predict_class("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3/Tomato___Spider_mites Two-spotted_spider_mite")
predict_classes("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3")

Score for Apple___Apple_scab: 93.18 %.
Done in 1.450 seconds for 44 images.
Score for Apple___Black_rot: 97.96 %.
Done in 1.371 seconds for 49 images.
Score for Apple___Cedar_apple_rust: 100.00 %.
Done in 0.810 seconds for 28 images.
Score for Apple___healthy: 96.00 %.
Done in 1.519 seconds for 50 images.
Score for Cherry_(including_sour)___Powdery_mildew: 100.00 %.
Done in 1.624 seconds for 51 images.
Score for Cherry_(including_sour)___healthy: 100.00 %.
Done in 1.598 seconds for 49 images.
Score for Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot: 83.33 %.
Done in 1.354 seconds for 42 images.
Score for Corn_(maize)___Common_rust_: 98.11 %.
Done in 1.725 seconds for 53 images.
Score for Corn_(maize)___Northern_Leaf_Blight: 91.89 %.
Done in 1.212 seconds for 37 images.
Score for Corn_(maize)___healthy: 98.21 %.
Done in 1.783 seconds for 56 images.
Score for Grape___Black_rot: 100.00 %.
Done in 1.145 seconds for 36 images.
Score for Grape___Esca_(Black_Measles): 92.16 %.
Done in 1.6

In [ ]:
def f(n, start_a=0, start_b=1):
  a, b = start_a, start_b
  for i in range(n-1):
    c = a + b
    a, b = b, c
  return b

f(3, "ha", "hi")

'hihahi'